In [347]:
from bs4 import BeautifulSoup
import urllib.request as Request
import csv
import time
def getXpath(node):
    expath = ""
    
    for parent in node.parents:
        expath = parent.name + "/" + expath
    
    expath = expath + node.name
    
    expath = expath.replace("[document]", "")
    
    return expath

def resultReceta(url):
    receta = []
    pagString = Request.urlopen(url).read()
    pagSoup = BeautifulSoup(pagString,"lxml")
    pagLxml = html.fromstring (pagString)
    nodo1 = pagSoup.find_all("h2", class_="subbuzz_name") [0]
    titulo = pagLxml.xpath(getXpath(nodo1) + "/text()")[0]
    receta.append(titulo)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_text buzz-superlist-item-text  buzz_superlist_item_wide   "]/div[@class="solid"]')
    nodo2 = nodos[0]
    nodoIngredients= nodo2.getchildren () [1:]
    ingreStr = ""
    for ig in nodoIngredients:
        ingreStr = ingreStr + ig.text_content().replace("\n","|") + "|"
    receta.append(ingreStr)
    nodoPreparacion = nodos[1].getchildren () [1:]
    prepStr = ""
    for prep in nodoPreparacion:
        prepStr = prepStr + prep.text_content ().replace("\n","|") + "|"
    receta.append(prepStr)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  "]/div[@class="solid"]/div[@class="sub_buzz_content"]/img')
    receta.append(nodos[0].attrib["rel:bf_image_src"])
    nodos = pagLxml.xpath ('//div[@class="video-embed-big video-embed-area bf_dom"]')
    receta.append(nodos[0].attrib["rel:thumb"])
    receta.append(nodos[0].attrib["rel:bf_bucket_data"])
    return receta

In [348]:
def downUrl(url):
    urlTools = []
    pagString = Request.urlopen(url).read()
    pagLxml = html.fromstring (pagString)
    nodos = pagLxml.xpath('//div[@class="card xs-relative xs-mb1 js-feed-item"]/div[@class="card--video"]/div[@class="xs-p2 clearfix card-title"]/a[@class="xs-col-11 link-gray"]')
    for n in nodos:
        urlTools.append([n.find("h2").text_content(), n.attrib["href"]])
    return urlTools

In [349]:
links = downUrl("https://www.buzzfeed.com/tasty/weeknights")
links = links + downUrl("https://www.buzzfeed.com/tasty/cheatday")
links = links + downUrl("https://www.buzzfeed.com/tasty/litebites")
links = links + downUrl("https://www.buzzfeed.com/tasty/junior")
links = links + downUrl("https://www.buzzfeed.com/tasty/vegetarian")
links = links + downUrl("https://www.buzzfeed.com/tasty/happyhour")

In [355]:
print("El total de recetas es: " + str(len(links)))
for l in links:
    print(l[0] + " --- " + l[1])

El total de recetas es: 101
One-Pan Garlic Parmesan Chicken And Vegetables --- https://www.buzzfeed.com/camillebergerson/heres-an-easy-dinner-that-will-leave-you-with-more-time-to-c?bffbtasty&ref=bffbtasty&utm_term=.coVaG6xoo#.aoONwVrpp
Rotini Pasta 4 Ways --- https://www.buzzfeed.com/alvinzhou/heres-four-ways-you-can-enjoy-rotini-pasta-this-week-for-lun?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw
Chicken And Asparagus Pasta --- https://www.buzzfeed.com/robertbroadfoot/chicken-penne?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw
Grilled Citrus Salmon And Asparagus --- https://www.buzzfeed.com/clairenolan/keep-it-fresh-on-the-grill-this-weekend-with-this-citrus-sal?bffbtasty&utm_term=4ldradw#4ldradw
Grilled Salmon With Avocado Salsa --- https://www.buzzfeed.com/robertbroadfoot/salmon-salsa-fun#.aoMdLJvE9
Vietnamese-Style Banh Mi Meatball Sandwich --- https://www.buzzfeed.com/emilyhorng/viet-ball#.vcP0RQGz7
Chicken Fajita Salad --- http://www.buzzfeed.com/alvinzhou/this-chicken-fa

In [359]:
import sys
errors = []
with open('tasty.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for l in links:
        try:
            receta = resultReceta(l[1])
            writer.writerow([l[0]] + receta)
        except:
            e = sys.exc_info()[0]
            errors.append(l + [str(e)])
        finally:
            time.sleep( 2 )


In [360]:
print(str(len(errors)))

24


In [363]:
with open('tasty.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    i = 1
    for row in reader:
        print(str(i) + " --- " + row[0])
        i = i + 1

1 --- One-Pan Garlic Parmesan Chicken And Vegetables
2 --- Chicken And Asparagus Pasta
3 --- Grilled Citrus Salmon And Asparagus
4 --- Grilled Salmon With Avocado Salsa
5 --- Vietnamese-Style Banh Mi Meatball Sandwich
6 --- Chicken Fajita Salad
7 --- Peking Duck-Inspired Burritos
8 --- Net Pancakes (Roti Jala)
9 --- Easy Chicken Piccata
10 --- Chocolate Chip Banana Monkey Bread 🍌
11 --- Peanut Butter-Stuffed Skillet Cookie
12 --- Chocolate Pretzel Pie
13 --- Blueberry Slab Pie
14 --- Tiramisu Mille Crepe
15 --- Cookie Dough Ice Cream Bites
16 --- Grilled Berry Cobbler
17 --- Honey Mustard Chicken Tater Tot Nachos
18 --- Chocolate Cream Cheese Croissant Bake
19 --- Peanut Butter Brownie Cookies
20 --- Peanut Butter Potato Chip Truffles
21 --- Is It Tasty?: Avocado Brownies Taste Test
22 --- Chicken And Asparagus Pasta
23 --- Grilled Citrus Salmon And Asparagus
24 --- Freezer-Prep Fruit Smoothies 4 Ways 🍓🍍🍌
25 --- Dark Chocolate Apricot Energy Bars
26 --- Creamy Chipotle Chicken Lettuce 

In [366]:
for e in errors:
    #print(e[0] + " --- " + e[1] + " --- " + e[2])
    print(e[1])

https://www.buzzfeed.com/alvinzhou/heres-four-ways-you-can-enjoy-rotini-pasta-this-week-for-lun?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw
http://www.buzzfeed.com/caroltan/this-is-how-you-make-black-bean-burgers#.ybAdbGBYN
https://www.buzzfeed.com/camillebergerson/these-amazing-no-bake-desserts-have-only-three-ingredients-s?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw
https://www.buzzfeed.com/alvinzhou/these-churro-ice-cream-sandwiches-will-make-your-date-night?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw
http://bzfd.it/2bh7xrl
https://www.buzzfeed.com/robertbroadfoot/heres-four-delicious-brazilian-truffle-recipes-you-have-to-t?bffbtasty&ref=bffbtasty&utm_term=.yh703BJRR#.amrXPD6kk
https://www.buzzfeed.com/alvinzhou/youre-going-to-want-to-prepare-a-dessert-feast-after-reading
https://www.buzzfeed.com/hannahmars/apple-pie-desserts?bffbtasty&utm_term=4ldradw#4ldradw
https://www.buzzfeed.com/hannahmars/cheesecakes-you-need-in-your-life
https://www.buzzfeed.com/buzzfeed/_draf